# LLMController Playground

This notebook demonstrates how to use the LLMController class with Claude and test various LangChain features.

## 1. Setup and Installation

Install required packages 
!pip install langchain anthropic openai python-dotenv langchain-community

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
from typing import Dict, Any, Optional, Union, List

In [2]:
# Load environment variables
load_dotenv()

print("Environment loaded successfully!")
print(f"Anthropic API Key loaded: {'Yes' if os.getenv('ANTHROPIC_API_KEY') else 'No'}")
print(f"OpenAI API Key loaded: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")

Environment loaded successfully!
Anthropic API Key loaded: Yes
OpenAI API Key loaded: Yes


In [3]:
# Check LangChain version
try:
    import langchain
    print(f"LangChain version: {langchain.__version__}")
except:
    print("LangChain version not available")

LangChain version: 0.3.17


## 2. Import LLMController Class

*Note: Make sure you have the LLMController class from the previous artifact saved as `llm_controller.py`*


In [4]:
from llm_controller import LLMController

In [ ]:
# Initialize the LLMController with Claude
llm = LLMController(
    llm="claude-3-sonnet-20240229",  # You can change this to claude-3-haiku-20240307 for faster/cheaper
    provider="claude",
    temperature=0.5 #optional
)

print("LLMController initialized with Claude!")
print(f"Current model info: {llm.current_model_info}")

LLMController initialized with Claude!
Current model info: {'provider': 'claude', 'model': 'claude-3-sonnet-20240229', 'type': 'ChatAnthropic', 'langchain_structure': 'new'}


In [8]:
# Test basic invoke functionality
response = llm.invoke("Hello! Can you tell me a brief joke about programming?")
print("Claude's Response:")
print(response.content)

Claude's Response:
Sure, here's a brief programming joke:

Why did the programmer quit his job? Because he didn't get arrays.


In [25]:
# Test with message format (like ChatGPT conversation)

# Import message classes if not already available
try:
    from langchain_core.messages import HumanMessage, AIMessage
except ImportError:
    from langchain.schema import HumanMessage, AIMessage
    
messages = [
    HumanMessage(content="I'm learning Python. Can you explain what a list comprehension is?"),
]

response = llm.invoke(messages)
print("Claude's explanation of list comprehensions:")
print(response.content)

Claude's explanation of list comprehensions:
Sure, a list comprehension is a concise way to create a new list in Python by applying an expression to each item in an iterable (like a list, tuple, or string). It provides a shorter syntax compared to using a for loop or a lambda function.

The basic syntax for a list comprehension is:

```python
new_list = [expression for item in iterable]
```

Here's an example that creates a list of squares from a list of numbers:

```python
numbers = [1, 2, 3, 4, 5]
squares = [x**2 for x in numbers]
print(squares)  # Output: [1, 4, 9, 16, 25]
```

You can also add a condition to filter the items in the new list:

```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = [x for x in numbers if x % 2 == 0]
print(evens)  # Output: [2, 4, 6, 8, 10]
```

List comprehensions can be more compact and readable than using for loops or lambda functions, especially for simple operations. However, for more complex operations, a regular for loop might be more read

In [26]:
# Test LangChain Prompt Templates
try:
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    print("✓ Using new prompt imports")
except ImportError:
    from langchain.prompts import ChatPromptTemplate
    from langchain.schema import StrOutputParser
    print("✓ Using legacy prompt imports")

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding tutor. Explain concepts clearly with examples."),
    ("human", "Explain {concept} in Python with a simple example.")
])

# Create a chain
chain = prompt | llm | StrOutputParser()

# Test the chain
result = chain.invoke({"concept": "decorators"})
print("Chain result - Python Decorators:")
print(result)

✓ Using new prompt imports
Chain result - Python Decorators:
Sure, I'll explain decorators in Python with a simple example.

In Python, a decorator is a function that takes another function as an argument, adds some functionality to it, and returns a new function. Decorators are used to modify the behavior of a function without changing its source code. They provide a way to wrap one function with another function to extend its functionality.

Here's a simple example to illustrate the concept of decorators:

```python
def uppercase_decorator(func):
    def wrapper():
        result = func()
        return result.upper()
    return wrapper

def say_hello():
    return "hello"

# Decorate the say_hello function with uppercase_decorator
say_hello = uppercase_decorator(say_hello)

print(say_hello())  # Output: HELLO
```

Let's break down this example:

1. We define a decorator function `uppercase_decorator` that takes a function `func` as an argument.
2. Inside `uppercase_decorator`, we de

In [27]:
# Test streaming functionality
print("Streaming response from Claude:")
print("-" * 50)

for chunk in llm.stream("Write a short poem about artificial intelligence"):
    print(chunk.content, end="", flush=True)

print("\n" + "-" * 50)

Streaming response from Claude:
--------------------------------------------------
Here's a short poem about artificial intelligence:

Artificial Intelligence, a marvel of our age,
Circuits and codes, a digital sage.
Algorithms weave, data's tapestry unfurled,
Unveiling insights, shaping our world.

Machines that learn, evolving with each task,
Patterns discerned, solutions they unmask.
From chatbots' wit to self-driving cars,
AI's potential reaches to the stars.

Yet, as we embrace this technological might,
Ethics must guide us, lest we lose sight.
For in these bytes, a power profound,
Humanity's values must be firmly bound.
--------------------------------------------------


In [34]:
# Test switching between different Claude models
test_question = "What is the capital of France?"
models_to_test = [
    "claude-3-haiku-20240307",    # Fastest, cheapest
    "claude-3-sonnet-20240229",   # Balanced
    "claude-3-opus-20240229"      # Most capable (if you have access)
]

print("Testing different Claude models:")
print("=" * 60)

for model in models_to_test:
    try:
        llm.switch_model(llm=model, provider="claude")
        response = llm.invoke(test_question)
        print(f"\n{model}:")
        print(f"Response: {response.content}")
    except Exception as e:
        print(f"\n{model}: Error - {e}")

Testing different Claude models:

claude-3-haiku-20240307:
Response: The capital of France is Paris.

claude-3-sonnet-20240229:
Response: The capital of France is Paris.

claude-3-opus-20240229:
Response: The capital of France is Paris.


In [33]:
#Advanced LangChain Features - Memory

try:
    from langchain.memory import ConversationBufferMemory
    from langchain_core.messages import HumanMessage, AIMessage
    print("✓ Memory imports successful")
except ImportError:
    from langchain.schema import HumanMessage, AIMessage
    print("✓ Using basic message imports")
    # Memory might not be available, so we'll implement simple history

# Create a simple conversation memory
conversation_history = []

def chat_with_memory(user_input):
    """Simple chat function with memory"""
    # Add user message
    conversation_history.append(HumanMessage(content=user_input))
    
    # Get response from Claude
    response = llm.invoke(conversation_history)
    
    # Add AI response to history
    conversation_history.append(response)
    
    return response.content

# Test conversation with memory
print("Conversation with memory:")
print("-" * 30)

response1 = chat_with_memory("My name is Alice and I love Python programming.")
print(f"User: My name is Alice and I love Python programming.")
print(f"Claude: {response1}")
print()

response2 = chat_with_memory("What's my name and what do I love?")
print(f"User: What's my name and what do I love?")
print(f"Claude: {response2}")

✓ Memory imports successful
Conversation with memory:
------------------------------
User: My name is Alice and I love Python programming.
Claude: It's great to meet you, Alice! It's wonderful that you have a passion for Python programming. Python is a versatile and beginner-friendly language that is widely used for various applications, including web development, data analysis, machine learning, and more.

As an AI language model, I can assist you with any questions or challenges you may encounter while learning or working with Python. Feel free to ask me about Python concepts, syntax, libraries, or any specific problems you're trying to solve.

To get started, could you tell me a bit more about your experience with Python so far? Are you just beginning your Python journey, or do you have some prior knowledge of the language? Knowing your current level will help me provide you with the most relevant information and guidance.

User: What's my name and what do I love?
Claude: Your name 

In [30]:
# Test LangChain Tools (Simple Example)

# import libraries
try:
    # Try new LangChain structure first
    from langchain_core.tools import BaseTool
    from pydantic import BaseModel, Field
    print("✓ New tool imports successful")
    TOOLS_AVAILABLE = True
except ImportError:
    try:
        # Try legacy structure
        from langchain.tools import BaseTool
        from pydantic import BaseModel, Field
        print("✓ Legacy tool imports successful")
        TOOLS_AVAILABLE = True
    except ImportError:
        print("⚠️ Tool imports not available - using function-based approach")
        BaseTool = None
        BaseModel = None
        Field = None
        TOOLS_AVAILABLE = False

# Create a simple custom tool
if BaseTool and BaseModel:
    class CalculatorInput(BaseModel):
        expression: str = Field(description="Mathematical expression to evaluate")

    class CalculatorTool(BaseTool):
        # Properly annotate all fields for Pydantic v2 compatibility
        name: str = "calculator"
        description: str = "Useful for mathematical calculations"
        args_schema: type[BaseModel] = CalculatorInput
        
        def _run(self, expression: str) -> str:
            try:
                # Use a safer eval alternative for production
                import ast
                import operator
                
                # Simple math operations mapping
                ops = {
                    ast.Add: operator.add,
                    ast.Sub: operator.sub,
                    ast.Mult: operator.mul,
                    ast.Div: operator.truediv,
                    ast.Pow: operator.pow,
                    ast.USub: operator.neg,
                }
                
                def safe_eval(node):
                    if isinstance(node, ast.Constant):  # Numbers
                        return node.value
                    elif isinstance(node, ast.BinOp):  # Binary operations
                        return ops[type(node.op)](safe_eval(node.left), safe_eval(node.right))
                    elif isinstance(node, ast.UnaryOp):  # Unary operations
                        return ops[type(node.op)](safe_eval(node.operand))
                    else:
                        raise TypeError(f"Unsupported operation: {type(node)}")
                
                try:
                    # Parse and evaluate safely
                    tree = ast.parse(expression, mode='eval')
                    result = safe_eval(tree.body)
                    return f"The result is: {result}"
                except:
                    # Fallback to basic eval for simple expressions (be careful in production!)
                    result = eval(expression)
                    return f"The result is: {result}"
                    
            except Exception as e:
                return f"Error calculating: {e}"

    # Test the tool
    try:
        print("Testing tool usage with Claude:")
        calculator = CalculatorTool()
        calc_result = calculator._run("25 * 4 + 10")
        print(f"Calculator tool result: {calc_result}")

        # Ask Claude to use the result
        response = llm.invoke(f"I calculated 25 * 4 + 10 and got: {calc_result}. Can you verify this is correct?")
        print(f"Claude's verification: {response.content}")
        
    except Exception as e:
        print(f"Tool creation failed with error: {e}")
        print("This is likely due to LangChain version compatibility issues.")
        print("Let's try a simpler approach...")
        
        # Simple function-based approach without BaseTool
        def simple_calculator(expression: str) -> str:
            try:
                result = eval(expression)  # Note: Use a safer parser in production
                return f"The result is: {result}"
            except Exception as e:
                return f"Error calculating: {e}"
        
        print("Using simple function approach:")
        calc_result = simple_calculator("25 * 4 + 10")
        print(f"Calculator result: {calc_result}")
        
        response = llm.invoke(f"I calculated 25 * 4 + 10 and got: {calc_result}. Can you verify this is correct?")
        print(f"Claude's verification: {response.content}")

else:
    print("Skipping tool example due to import issues")
    print("This is normal in newer LangChain versions where agent structure has changed")
    
    # Alternative: Simple function-based tool demonstration
    print("\nUsing simple function approach instead:")
    
    def calculator_function(expression: str) -> str:
        """Simple calculator function"""
        try:
            result = eval(expression)  # Note: Use a safer math parser in production
            return f"The result is: {result}"
        except Exception as e:
            return f"Error calculating: {e}"
    
    # Test the function
    calc_result = calculator_function("25 * 4 + 10")
    print(f"Calculator function result: {calc_result}")
    
    # Ask Claude about it
    response = llm.invoke(f"I used a calculator function and got: {calc_result}. Is this calculation correct?")
    print(f"Claude's response: {response.content}")

✓ New tool imports successful
Testing tool usage with Claude:
Calculator tool result: The result is: 110
Claude's verification: I'm sorry, but the result you provided is not correct. Let's solve this problem step by step to find the correct answer.

Given:
- The problem is to calculate 25 * 4 + 10

Step 1: Perform the multiplication.
25 * 4 = 100

Step 2: Add 10 to the result of the multiplication.
100 + 10 = 110

Therefore, the correct result of the expression 25 * 4 + 10 is 110.

Your calculation was correct, but it seems like there was a misunderstanding in your statement. The result you provided (110) is indeed the correct answer to the given problem.


In [ ]:
# Test Switch to Different Providers (if available)

# %%
# Test switching to OpenAI if available
if os.getenv("OPENAI_API_KEY"):
    print("Testing provider switching:")
    print("-" * 30)
    
    # Claude response
    llm.switch_model(llm="claude-3-sonnet-20240229", provider="claude")
    claude_response = llm.invoke("What's the capital of France?")
    print(f"Claude: {claude_response.content}")
    
    # Switch to OpenAI
    llm.switch_model(llm="gpt-3.5-turbo", provider="openai")
    openai_response = llm.invoke("What's the capital of France?")
    print(f"OpenAI: {openai_response.content}")
    
    # Switch back to Claude
    llm.switch_model(llm="claude-3-sonnet-20240229", provider="claude")
    print(f"Switched back to: {llm.current_model_info}")
else:
    print("OpenAI API key not available - skipping provider switching test")

In [35]:
from llm_controller import AdaptiveLLM

adaptive = AdaptiveLLM()

creative_response = adaptive.query("Write a poem about AI", "creative")
print("Creative Response:")
print(creative_response.content)


Creative Response:
Here's a poem about AI:

Artificial Intelligence, a marvel of our time,
A creation of code, a symphony sublime.
Algorithms dance with data's vast embrace,
Unraveling patterns, leaving no trace.

Neural networks intertwine, a labyrinth of thought,
Processing knowledge that can't be bought.
Machine learning's evolution, a ceaseless quest,
Adapting, growing, forever blessed.

From language processing to image recognition,
AI's prowess knows no restriction.
Automating tasks, enhancing human life,
Cutting through complexity's knife.

Yet, amidst its brilliance, a lingering fear,
Of ethical boundaries we must hold dear.
A delicate balance, a dance we must lead,
Harnessing AI's power, while planting wisdom's seed.

Embrace the future, where AI reigns,
But let humanity's values guide the reins.
For in this symbiosis, our true strength lies,
Merging technology and wisdom's wise eyes.


In [36]:
# Final Test - Complex Chain

llm = LLMController(
    llm="claude-3-sonnet-20240229",  # You can change this to claude-3-haiku-20240307 for faster/cheaper
    provider="claude"
)

try:
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
except ImportError:
    from langchain.prompts import ChatPromptTemplate
    from langchain.schema import StrOutputParser

# Create a complex chain for code review
code_review_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert Python code reviewer. Provide constructive feedback."),
    ("human", """
    Please review this Python code and provide feedback:
    
    ```python
    {code}
    ```
    
    Focus on:
    1. Code quality
    2. Best practices
    3. Potential improvements
    4. Any bugs or issues
    """)
])

# Sample code to review
sample_code = """
def calculate_average(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total / len(numbers)

result = calculate_average([1, 2, 3, 4, 5])
print(result)
"""

# Create and run the chain
review_chain = code_review_prompt | llm | StrOutputParser()
review_result = review_chain.invoke({"code": sample_code})

print("Claude's Code Review:")
print("=" * 50)
print(review_result)

Claude's Code Review:
The provided Python code calculates the average of a list of numbers. Here's my feedback based on the requested areas:

1. **Code Quality**:
   - The code is readable and follows a straightforward approach.
   - The function name `calculate_average` is descriptive and accurately represents its purpose.
   - The code uses appropriate variable names (`total` and `i`).

2. **Best Practices**:
   - The code follows the Python style guide (PEP 8) for naming conventions and code formatting.
   - The use of a loop to iterate over the list and calculate the sum is a common and acceptable practice.
   - The code uses the built-in `len()` function to determine the length of the list, which is a good practice.

3. **Potential Improvements**:
   - Instead of using a traditional `for` loop with `range(len(numbers))`, you could use the more Pythonic way of iterating over the list directly:

     ```python
     def calculate_average(numbers):
         total = sum(numbers)
      